## Explanatory Notebook


*Notebook for **Risk-Aware Reinforcement Learning for Goal-Based Portfolio Optimisation**, by Mathew Cater Benavides and Sebastian Jaimungal*,

This Jupyter notebook showcases how to use the Python files for using our proposed deep policy gradient algorithm and reproducing some of the main figures in the experiments section. All the Python code and modules are publicly available in the Github repository.

### 1. Importation of Python Libraries

We load some basic Python libraries, such as NumPy, PyTorch, SciPy and Matplotlib.

In [ ]:
# numpy
import numpy as np

# plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import gaussian_kde

# pytorch
import torch as T

# misc.
from time import time
from tqdm import tqdm
from torch.nn import ReLU
np.seterr(divide='ignore', invalid='ignore')
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Palatino"
})


We as well load the relevant classes and functions in our Python modules and refer the user to the Python files for a description of all functions in the modules. Furthermore, the user can modify these files to consider non-investigated market models or risk measures.

In [ ]:
from utils import cmap, colors # colors
from hyperparams import init_params, print_params # hyperparameters
from models import PolicyANN # ANN structures
from envs import BS_Environment, FactorModel_Environment # environment
from agents import Agent # deep policy gradient algorithm

### 2. Initialisation of Training Hyperparameters

In [ ]:
market_model = "BS" # specify the market model
num_assets = 4
env_params = {'num_assets': num_assets, # number of total assets in market (risk-free and risky)
                'drifts': T.tensor([0.03, 0.06, 0.09]), # drifts of risky assets
                'vols': T.tensor([0.06, 0.12, 0.18]), # volatilities of risky assets
                'corr_matrix': T.tensor([[1.0, 0.2, 0.2], # correlation structure of risky asset prices
                                         [0.2, 1.0, 0.2],
                                         [0.2, 0.2, 1.0]]), 
                'interest_rate': 0.025, # interest rate of risk-free asset                
                'goal_prob': 0.0, # confidence level of meeting returns requirement 
                'returns_req': 0.0, # returns requirement for goal
                'alpha': 0.80,
                'beta': 0.9, 
                'q': 1.0, 
                'phi': 0.0, # transaction costs
                'T': 1.0, # trading horizon
                'Ndt': 1, # number of periods
                'init_wealth': 1.0, # initial wealth
                'S0': T.tensor([1.0, 1.0, 1.0]) # initial risky asset prices
                }
algo_params = {'num_epochs' : 1_000, # number of iterations of entire training loop
                'batch_size' : 750, # mini-batch size for gradient estimates
                'num_layers': 10, # number of layer in policy network
                'hidden_size': 16, # width of hidden layers of policy network
                'learn_rate': 0.001, # learning rate of policy network
                'init_lamb' : 1.0, # initial Lagrange multiplier    
                'init_mu' : 10.0, # initial penalty strength
                'pen_strength_lr' : 1.5, # penalty strength learning rate
                'pen_update_freq' : 5, # Lagrange multiplier and penalty strength update frequency
                } 
print_params(env_params, algo_params, market_model)

We instantiate the environment from the envs.py file to get functions to interact with the data-generating processes.

In [ ]:
env = BS_Environment(env_params)

### 3. Training of Risk-Aware Agents & Initialisation of Testing Parameters

We initialise the neural network structure for our algorithm from the **models.py** file, we as well instantiate our risk-aware agent from the *agents.py* file with the aforementioned policy, environment and algorithm specifications. The $\texttt{Agent()}$ class regroups the different routines to solve our optimisation problem. We have respectively:

- an ANN representing the policy $\pi^{\theta}$;
- an $\texttt{Agent()}$ object representing the risk-aware agent with the aforementioned specifications.

In [ ]:
policy = PolicyANN(env, algo_params)
agent = Agent(env=env, # environment
              algo_params=algo_params, # algorithm specifications 
              policy=policy) # policy

We next start the training phase of our deep policy gradient algorithm -- this corresponds to the Algorithm 5.1 presented in our paper. We as well set up hyperparameters for the testing phase, in particular, the number of episodes and the seed for the random number generator. 

In [ ]:
T.random.manual_seed(54321) # set seed for replication purposes
Nsimulations = 10_000 # number of simulations following the optimal strategy

In [ ]:
# time the algorithm
start_time = time()

for epoch in tqdm(range(agent.algo_params["num_epochs"])):
            
    # at final epoch of training, store prob. of meeting goal, dist. of term. wealth and action hist. over final mini-batch
    if epoch == agent.algo_params["num_epochs"] - 1:
        agent.algo_params["batch_size"] = Nsimulations
        
        # update policy     
        _, RDEU, return_prob, term_wealth_samps, pi = agent.update_policy() 
        
        # store RDEU to track training
        agent.update_history(RDEU, return_prob) 
        
        # store store prob. of meeting goal, dist. of term. wealth and action hist.
        agent.term_wealth_dist = term_wealth_samps 
        agent.return_prob = return_prob
        agent.position_hist = pi
    
    else:
        # update policy     
        _, _, return_prob, term_wealth_samps, pi = agent.update_policy() 
        
        # update Lagrange multipliers
        if epoch % agent.algo_params["pen_update_freq"] == 0:
            agent.update_multipliers(constr_err=ReLU()(agent.env.params["goal_prob"] - return_prob))    
            
    start_time = time()


### 4. Testing of Risk-Aware Agents


##### 4.1 Distribution of Terminal Wealth

We investigate the terminal wealth distribution when the agent adheres to the learned optimal policy. To accomplish this, we execute a large batch of complete episodes in accordance with the learned policy. We as well report the probability that the agent achieves their desired realised gain.

In [ ]:
(fig, ax) = plt.subplots(1, 3, figsize=(18,5))  \
                            
# plot RDEU through training
ax[0].plot(np.arange(algo_params["num_epochs"]) + 1, 
                     T.stack(agent.RDEU_history)[:-1], 
                     color=colors[0])
ax[0].set(xlabel='Epochs', ylabel='RDEU', title='RDEU by Epoch')    
ax[0].grid()

# plot terminal wealth distribution of learned strategy using Nsims full episodes
term_wealth = agent.term_wealth_dist.detach().squeeze()
mean_term_wealth, std_term_wealth = T.mean(term_wealth), T.std(term_wealth)
num_bins = 50
domain = np.linspace(mean_term_wealth - 5*std_term_wealth, mean_term_wealth + 5*std_term_wealth, 1500)
term_wealth_kde = gaussian_kde(term_wealth)
ax[1].hist(term_wealth, density=True, bins=num_bins, color=colors[0], alpha=0.5)
ax[1].set(title=r'Distribution of Terminal Wealth', xlabel=r'Terminal Wealth: $X^{\theta}$', ylabel='Density')
ax[1].plot(domain, term_wealth_kde(domain), color=colors[0])

# plot probability of posting desired return through training
ax[2].plot(np.arange(algo_params["num_epochs"]) + 1,
                     T.stack(agent.constraint_prob_history)[:-1],
                     color=colors[0])
ax[2].set(xlabel='Epochs', ylabel=r'$\mathbb{P}(X^\theta \geq (1+c)X_0)$', title='Probability of Success by Epoch')    
ax[2].grid()

# check if constraint satisfied
c = env_params["returns_req"]
print(f'Probability of return exceeding {100*c}%: {str(round(float(100*agent.return_prob), 3))}%')
fig.tight_layout()

##### 4.2 Strategy Analysis

We observe the position paths in each of the assets over the 10,000 full testing episodes and plot their quantiles to investigate the learned strategy. 

In [ ]:
position_hist = agent.position_hist.detach().squeeze().numpy()

if env_params['Ndt'] == 1:
    
    # figure parameters
    nrows = 1
    ncols = num_assets
    fig, axes = plt.subplots(nrows, ncols, sharey='all', sharex='all', figsize=(15,7))
    grid = plt.GridSpec(nrows, ncols)
    labels = [r'$\pi_{\theta}^{(0)}$', r'$\pi_{\theta}^{(1)}$', r'$\pi_{\theta}^{(2)}$', r'$\pi_{\theta}^{(3)}$']

    for idx_asset in range(num_assets):
        temp = axes[idx_asset].imshow(position_hist[0][idx_asset].reshape(1,1),
                                interpolation='none',
                                cmap=cmap,
                                aspect='auto',
                                vmin=np.array(0.0),
                                vmax=np.array(1.0)
                                )
        axes[idx_asset].set_xticks([])
        axes[idx_asset].set_yticks([])
        axes[idx_asset].set_title(f'Allocation Asset {idx_asset}: {labels[idx_asset]}', fontsize=15)

    fig.colorbar(temp, orientation='horizontal', ax=axes, shrink=0.8, pad=0.2)
    fig.suptitle('Learned Allocations', fontsize=19)
    
else:
    
    fig, axs = plt.subplots(1, env_params["num_assets"], figsize=(18,5), sharey=True)
    position_hist = agent.position_hist.detach().squeeze()
    periods = T.arange(env_params["Ndt"])
    # quantile 
    q = 0.5

    #labels
    labels = [r'$\pi_{\theta}^{(0)}$', r'$\pi_{\theta}^{(1)}$', r'$\pi_{\theta}^{(2)}$', r'$\pi_{\theta}^{(3)}$']

    for j in range(env.params["num_assets"]):
        axs[j].plot(np.quantile(position_hist[:,:,j], axis=0, q=q), color=colors[0], label=f'q={q}') # plot quantile
        axs[j].plot(position_hist[0,:,j], alpha=0.5, color=colors[0]) # plot position paths beneath 
        axs[j].set(title=f'Allocations \& Quantile: Asset {j}', xlabel='Period', ylabel=labels[j])
        axs[j].legend()
        
    fig.tight_layout()

In [ ]:
position_hist

In [ ]:
# CVaR thresholds considered:
alphas = np.array([0.01, 0.75, 0.85, 0.99])

# figure parameters
nrows = num_assets
ncols = len(alphas)
fig, axes = plt.subplots(nrows, ncols, sharey='all', sharex='all', figsize=(10,7))

# Instantiate one agent, then transfer-learn optimal policy for each threshold.
agent = Agent(env, algo_params)

row_titles = [r'$\pi_{\theta}^{(0)}$',
              r'$\pi_{\theta}^{(1)}$',
              r'$\pi_{\theta}^{(2)}$',
              r'$\pi_{\theta}^{(3)}$']
column_titles = [f'CVaR: {alpha}' for alpha in alphas]

for alpha_idx, alpha_val in enumerate(alphas):
    
    agent.env.params['alpha'] = alpha_val 
    agent.Train()
    position_hist = agent.position_history.detach().squeeze().numpy()

    for idx_asset in range(num_assets):
        
        temp = axes[idx_asset, alpha_idx].imshow(position_hist[0][idx_asset].reshape(1,1),
                                                 interpolation='none',
                                                 cmap=cmap,
                                                 aspect='auto',
                                                 vmin=np.array(0.0),
                                                 vmax=np.array(1.0)
                                                )
        axes[idx_asset, alpha_idx].set_xticks([])
        axes[idx_asset, alpha_idx].set_yticks([])
        
        if alpha_idx == ncols - 1:
            axes[idx_asset, alpha_idx].set_ylabel(row_titles[idx_asset],
                                                  rotation='horizontal',
                                                  fontsize=15,
                                                  fontweight='semibold',
                                                  labelpad=18)
            axes[idx_asset, alpha_idx].yaxis.set_label_position("right")

        if idx_asset == 0:
            axes[idx_asset, alpha_idx].set_title(column_titles[alpha_idx] + '\n',
                                                 rotation='horizontal',
                                                 fontsize=15,
                                                 fontweight='semibold')

fig.colorbar(temp, ax=axes, orientation='horizontal', shrink=0.8, pad=0.1)

# fig.tight_layout()

##### 4.3 Varying the Goal

We investigate how the distribution of the terminal wealth evolves as the goal probability increases: $p = 0.5, 0.75, 0.80, 0.95$.

In [ ]:
goals = [0.5, 0.75, 0.80, 0.95]
agents = []

for goal in goals:
    params = env_params
    params["goal_prob"] = goal
    env = BS_Environment(params)
    agent = Agent(env, algo_params)
    agent.Train(Nsims=Nsimulations)
    agents.append(agent)

(fig, ax) = plt.subplots(1, 3, figsize=(18,5))  \

for i, agent in enumerate(agents):                        
    # plot RDEU through training
    ax[0].plot(np.arange(algo_params["num_epochs"]) + 1, 
               T.stack(agent.RDEU_history), 
               color=colors[i],
               label=f'p = {goals[i]}')
    ax[0].set(xlabel='Epochs', 
              ylabel='RDEU',
              title='RDEU by Epoch')    
    ax[0].grid()

    # plot terminal wealth distribution of learned strategy using 10,000 full episodes
    term_wealth = agent.term_wealth_dist.detach().squeeze()
    mean_term_wealth, std_term_wealth = T.mean(term_wealth), T.std(term_wealth)
    num_bins = 50
    domain = np.linspace(mean_term_wealth - 5*std_term_wealth,
                         mean_term_wealth + 5*std_term_wealth, 
                         1500)
    term_wealth_kde = gaussian_kde(term_wealth)
    ax[1].hist(term_wealth, 
               density=True, 
               bins=num_bins, 
               color=colors[i], 
               alpha=0.5)
    ax[1].set(title=r'Distribution of Terminal Wealth',
              xlabel=r'Terminal Wealth: $X^{\theta}$', 
              ylabel='Density')
    ax[1].plot(domain, 
               term_wealth_kde(domain), 
               color=colors[i], 
               label=f'p = {goals[i]}')

    # plot probability of posting desired return through training
    ax[2].plot(np.arange(algo_params["num_epochs"]) + 1,
                        T.stack(agent.constraint_prob_history),
                        color=colors[i],
                        label=f'p = {goals[i]}')
    ax[2].set(xlabel='Epochs', 
              ylabel=r'$\mathbb{P}(X^\theta \geq (1+c)X_0)$', 
              title='Probability of Success by Epoch')    
    ax[2].grid()

    # check if constraint satisfied
    c = env_params["returns_req"]
    print(f'Probability of return exceeding {100*c}%: {str(round(float(100*agent.return_prob), 3))}%')
    fig.tight_layout()
    
ax[0].legend()
ax[1].legend()
ax[2].legend()